In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q findspark pyspark

In [8]:
# Inicializa Spark

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("GenerateRefinedMetadata") \
    .getOrCreate()

In [9]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [10]:
def generate_metadados(dataframe, df_name: str):
    """
    Gera metadados de `dataframe` e adiciona em todas as linhas
    uma coluna 'Warehouse' com o nome do DataFrame de origem.
    """
    # 1) coleta as estatísticas originais
    column_names  = dataframe.columns
    dtypes        = [dtype for _, dtype in dataframe.dtypes]
    total_count   = dataframe.count()
    null_counts   = [dataframe.filter(F.col(c).isNull()).count() for c in column_names]
    null_percents = [round((cnt/total_count)*100, 2) for cnt in null_counts]
    cardinality   = [dataframe.select(c).distinct().count() for c in column_names]

    # monta o DataFrame de metadados
    metadata = spark.createDataFrame(
        zip(column_names, dtypes, null_counts, null_percents, cardinality),
        schema=["nome_variavel", "tipo", "qt_nulos", "percent_nulos", "cardinalidade"]
    ).orderBy(F.desc("percent_nulos"))

    # Coluna de identificação
    metadata = metadata.withColumn("Warehouse", F.lit(df_name))

    # Warehouse é para que o nome do df seja a primeira coluna
    cols = ["Warehouse"] + [c for c in metadata.columns if c != "Warehouse"]
    return metadata.select(*cols)

In [11]:
refined_base = "/content/drive/MyDrive/PREVISAO_DEMANDA/REFINED"
out_base     = os.path.join(refined_base, "METADADOS_REFINED")

warehouses = ['J','A','S','C']
splits     = ['train','val']

for wh in warehouses:
    for split in splits:
        # 1) Lê o DataFrame refinado
        path = os.path.join(refined_base, f"FEATURES_{wh}", split)
        df   = spark.read.parquet(path)

        # 2) (Opcional) cache para acelerar
        df.cache().count()

        # 3) Gera metadados
        source_name = f"METADADOS_DF_{wh}_{split}"
        md = generate_metadados(df, source_name)

        # 4) Define saída em METADADOS_REFINED/METADADOS_DF_{wh}_{split}
        out_path = os.path.join(out_base, source_name)

        # 5) Grava em CSV com header (um arquivo)
        md.coalesce(1) \
          .write \
          .mode("overwrite") \
          .option("header", True) \
          .csv(out_path)

        print(f"✔ Metadados {source_name} salvos em {out_path}")

spark.stop()

✔ Metadados METADADOS_DF_J_train salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_J_train
✔ Metadados METADADOS_DF_J_val salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_J_val
✔ Metadados METADADOS_DF_A_train salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_A_train
✔ Metadados METADADOS_DF_A_val salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_A_val
✔ Metadados METADADOS_DF_S_train salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_S_train
✔ Metadados METADADOS_DF_S_val salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_S_val
✔ Metadados METADADOS_DF_C_train salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REFINED/METADADOS_DF_C_train
✔ Metadados METADADOS_DF_C_val salvos em /content/drive/MyDrive/PREVISAO_DEMANDA/REFINED/METADADOS_REF